In [1]:
import pathlib
import astropy.units as u
import astropy.time
import astropy.wcs
import astropy.io.fits


def split_cubes(
    path_source: list[str | pathlib.Path],
    directory_destination: str | pathlib.Path,
) -> list[str | pathlib.Path]:
    """
    Split fits files containing wavelength, x, and y axes into separate fits files
    containing only wavelength and y axes.
    
    Parameters
    ----------
    path_source
        A list of paths to source files to be converted
    directory_destination
        A directory in which to place the results.
    """
    
    directory_destination = pathlib.Path(directory_destination)
    directory_destination.mkdir(exist_ok=True, parents=True)
    
    for path in path_source:
        
        path = pathlib.Path(path)
        
        hdul = astropy.io.fits.open(path)
        
        index_main = 0
        index_aux1 = ~1
        index_aux2 = ~2
        index_image = slice(index_main + 1, index_aux1)
        
        hdu_main = hdul[index_main]
        hdu_image = hdul[index_image]
        hdu_aux1 = hdul[index_aux1]
        hdu_aux2 = hdul[index_aux2]
        
        num_steps = hdu_main.header["NEXP"]
        
        date_obs = astropy.time.Time(hdu_main.header["DATE_OBS"])
        time = date_obs + hdu_aux1.data[..., hdu_aux1.header["TIME"]] * u.s
        
        for step in range(num_steps):
                
            
            slice_step = slice(step, step + 1)
            
            hdu_step_main = hdu_main.copy()
            hdu_step_main.header["DATE_OBS"] = time[step].fits
            
            hdu_step_image = [hdu.copy() for hdu in hdu_image]
            for hdu in hdu_step_image:
                wcs = astropy.wcs.WCS(hdu.header)
                
                if step == 0:
                    print(f"{wcs.wcs.crval}")
                
                hdu.header.update(wcs[slice_step].to_header())
                hdu.data = hdu.data[slice_step]
                
            hdu_step_aux1 = hdu_aux1.copy()
            hdu_step_aux1.data = hdu_step_aux1.data[slice_step]
            
            hdu_step_aux2 = hdu_aux2.copy()
            
            hdul_step = [hdu_step_main] + hdu_step_image + [hdu_step_aux1, hdu_step_aux2]
            hdul_step = astropy.io.fits.HDUList(hdul_step)
            
            filename = directory_destination / f"{path.stem}_s{step}{path.suffix}"
            
            hdul_step.writeto(filename, overwrite=True)        

In [2]:
directory_source = pathlib.Path(r"C:\Users\byrdie\Kankelborg-Group\interface-region-imaging-spectrograph\iris_l2_20240222_214410_3680334922_raster")
path_source = sorted(list(directory_source.glob("*")))

split_cubes(
    path_source=path_source[:10],
    directory_destination=r"C:\Users\byrdie\Kankelborg-Group\interface-region-imaging-spectrograph\iris_l2_20240222_214410_3680334922_raster_split"
)

[ 1.33168977e-07  1.05176389e-01 -1.30635000e-01]
[ 1.38067031e-07  1.05176389e-01 -1.30635000e-01]
[ 2.78361509e-07  1.05176389e-01 -1.30635000e-01]
[ 1.33168977e-07  1.05177778e-01 -1.30632222e-01]
[ 1.38067031e-07  1.05177778e-01 -1.30632222e-01]
[ 2.78361509e-07  1.05177778e-01 -1.30632222e-01]
[ 1.33168977e-07  1.05190556e-01 -1.30610833e-01]
[ 1.38067031e-07  1.05190556e-01 -1.30610833e-01]
[ 2.78361509e-07  1.05190556e-01 -1.30610833e-01]
[ 1.33168977e-07  1.05181111e-01 -1.30621389e-01]
[ 1.38067031e-07  1.05181111e-01 -1.30621389e-01]
[ 2.78361509e-07  1.05181111e-01 -1.30621389e-01]
[ 1.33168977e-07  1.05188611e-01 -1.30604444e-01]
[ 1.38067031e-07  1.05188611e-01 -1.30604444e-01]
[ 2.78361509e-07  1.05188611e-01 -1.30604444e-01]
[ 1.33168977e-07  1.05194722e-01 -1.30598611e-01]
[ 1.38067031e-07  1.05194722e-01 -1.30598611e-01]
[ 2.78361509e-07  1.05194722e-01 -1.30598611e-01]
[ 1.33168977e-07  1.05189444e-01 -1.30591389e-01]
[ 1.38067031e-07  1.05189444e-01 -1.30591389e-01]


In [3]:
hdul = astropy.io.fits.open(r"C:\Users\byrdie\Kankelborg-Group\interface-region-imaging-spectrograph\iris_l2_20240209_011103_3650105103_raster\iris_l2_20240209_011103_3650105103_raster_t000_r00000.fits")

In [4]:
print(repr(hdul[0].header))

SIMPLE  =                    T / Written by IDL:  Thu Apr 11 11:22:44 2024      
BITPIX  =                   16 / Number of bits per data pixel                  
NAXIS   =                    0 / Number of data axes                            
EXTEND  =                    T / FITS data may contain extensions               
DATE    = '2024-04-11'         / Creation UTC (CCCC-MM-DD) date of FITS header  
COMMENT FITS (Flexible Image Transport System) format is defined in 'Astronomy  
COMMENT and Astrophysics', volume 376, page 359; bibcode 2001A&A...376..359H    
TELESCOP= 'IRIS    '           /                                                
INSTRUME= 'SPEC    '           /                                                
DATA_LEV=              2.00000 /                                                
LVL_NUM =              2.00000 /                                                
VER_RF2 = 'L12-2019-08-08'     /                                                
DATE_RF2= '2024-04-11T17:58:

In [5]:
print(hdul[~1].data[..., hdul[~1].header["TIME"]].shape)

(276,)


In [6]:
print(hdul[1].data.shape)

(276, 385, 169)


In [7]:
print(repr(hdul[1].header))

XTENSION= 'IMAGE   '           / IMAGE extension                                
BITPIX  =                  -32 / Number of bits per data pixel                  
NAXIS   =                    3 / Number of data axes                            
NAXIS1  =                  169 /                                                
NAXIS2  =                  385 /                                                
NAXIS3  =                  276 /                                                
PCOUNT  =                    0 / No Group Parameters                            
GCOUNT  =                    1 / One Data Group                                 
CDELT1  =      0.0259600002319 /                                                
CDELT2  =             0.332700 /                                                
CDELT3  =        0.00000000000 /                                                
CRPIX1  =              1.00000 /                                                
CRPIX2  =              193.0

In [8]:
hdul[~0].data[0]

('3700201130', '3700200362', '1176', '3700201130', '1224', '/irisa/data/level1/2024/02/09/H0100/iris20240209_01110418_fuv.fits', '/irisa/data/level1/2024/02/09/H0100/iris20240209_01110412_nuv.fits-57.0,-57.0,-56.0,-57.0,1.7,1.0,1.0,0.9,2.4,2.0,1.7,2.3-57.0,-57.0,-56.0,-57.0,1.7,1.0,1.0,0.9,2.4,2.0,1.7,2.3')